In [ ]:
#Importing necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json

# Display setup
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Load food data
food_df = pd.read_excel('/content/food-calories.xlsx')
print("✅ Food Data Loaded:")
display(food_df.head())

# Load activities (MET) data
activity_df = pd.read_excel('/content/MET-values.xlsx')
print("✅ Activities Data Loaded:")
display(activity_df.head())

✅ Food Data Loaded:


,ID,name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Serving Description 1 (g)
0,1,Apple,Fruits,52,0.2,0.3,14.0,100
1,2,Banana,Fruits,96,0.3,1.3,27.0,118
2,3,Chicken Breast,Meat,165,3.6,31.0,0.0,100
3,4,Broccoli,Vegetables,55,0.3,3.7,11.0,91
4,5,Rice,Grains,130,0.3,2.7,28.0,158


✅ Activities Data Loaded:


,ACTIVITY,SPECIFIC MOTION,METs
0,Walking,Strolling,2.0
1,Running,Jogging,7.0
2,Cycling,Leisure cycling,4.0
3,Swimming,Freestyle moderate,6.0
4,Yoga,Hatha yoga,2.5


In [ ]:
# Create a dictionary to hold users
users = {}

# Function to add a user
def add_user(user_id, name, age, weight, height, sex):
    users[user_id] = {
        'name': name,
        'age': age,
        'weight': weight,
        'height': height,
        'sex': sex
    }

# Example: Add a user
add_user('U001', 'Alex', 28, 70, 175, 'Male')

# View all users
print("Current Users:")
for uid, info in users.items():
    print(f"{uid}: {info}")


Current Users:
U001: {'name': 'Alex', 'age': 28, 'weight': 70, 'height': 175, 'sex': 'Male'}


In [ ]:
# Function to calculate BMR based on gender
def calculate_bmr(user):
    weight = user['weight']
    height = user['height']
    age = user['age']
    sex = user['sex'].lower()

    if sex == 'male':
        bmr = 66.4730 + (13.7516 * weight) + (5.0033 * height) - (6.7550 * age)
    elif sex == 'female':
        bmr = 655.0955 + (9.5634 * weight) + (1.8496 * height) - (4.6756 * age)
    else:
        raise ValueError("Sex must be 'Male' or 'Female'")

    return round(bmr, 2)


In [ ]:
alex_bmr = calculate_bmr(users['U001'])
print(f"BMR for {users['U001']['name']}: {alex_bmr} calories/day")


BMR for Alex: 1715.52 calories/day


In [ ]:
# Normalize column names
food_df.columns = food_df.columns.str.strip().str.lower().str.replace(' ', '_')
print("Food Data Columns:", food_df.columns.tolist())

Food Data Columns: ['id', 'name', 'food_group', 'calories', 'fat_(g)', 'protein_(g)', 'carbohydrate_(g)', 'serving_description_1_(g)']


In [ ]:
def calculate_food_calories(food_name, servings):
    row = food_df[food_df['name'] == food_name]
    if not row.empty:
        calories_per_serving = row['calories'].values[0]
        return round(calories_per_serving * servings, 2)
    else:
        return 0
food_options = food_df['name'].tolist()


In [ ]:
# Standardize activity column names
activity_df.columns = activity_df.columns.str.strip().str.lower().str.replace(' ', '_')

print("Activity Data Columns:", activity_df.columns.tolist())

Activity Data Columns: ['activity', 'specific_motion', 'mets']


In [ ]:
def calculate_activity_calories(activity_name, duration_minutes, weight_kg):
    row = activity_df[activity_df['activity'] == activity_name]
    if not row.empty:
        met = row.iloc[0]['mets']
        duration_hours = duration_minutes / 60
        return round(met * weight_kg * duration_hours, 2)
    else:
        return 0

activity_options = activity_df['activity'].unique().tolist()


In [ ]:
# Store logs with key as (user_id, date)
daily_logs = {}

def log_daily_info(user_id, date, food_log, activity_log):
    user = users.get(user_id)
    if not user:
        return f"User ID {user_id} not found."

    # Calculate BMR
    bmr = calculate_bmr(user)

    # Calculate total food calories
    total_food_cals = sum([calculate_food_calories(food, servings) for food, servings in food_log])

    # Calculate total activity calories
    total_activity_cals = sum([calculate_activity_calories(activity, duration, user['weight'])
                               for activity, duration in activity_log])

    # Calculate net calories
    net_cals = round(total_food_cals - bmr - total_activity_cals, 2)

    # Save in daily logs
    daily_logs[(user_id, date)] = {
        'food_log': food_log,
        'activity_log': activity_log,
        'bmr': bmr,
        'food_calories': total_food_cals,
        'activity_calories': total_activity_cals,
        'net_calories': net_cals
    }

    return daily_logs[(user_id, date)]


In [ ]:
import json

def save_logs_to_file(file_path='daily_logs.json'):
    # Convert tuple keys to strings for JSON compatibility
    serializable_logs = {f"{key[0]}|{key[1]}": value for key, value in daily_logs.items()}
    with open(file_path, 'w') as f:
        json.dump(serializable_logs, f, indent=4)
    print("✅ Daily logs saved to", file_path)


In [ ]:
def load_logs_from_file(file_path='daily_logs.json'):
    global daily_logs
    try:
        with open(file_path, 'r') as f:
            loaded_data = json.load(f)
            # Convert string keys back to tuple format
            daily_logs = {tuple(key.split('|')): value for key, value in loaded_data.items()}
        print("✅ Daily logs loaded from", file_path)
    except FileNotFoundError:
        print("⚠️ No existing log file found.")


In [ ]:
!pip install ipywidgets --quiet

import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime, timedelta


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.0 MB/s eta 0:00:00


In [ ]:
user_selector = widgets.Dropdown(
    options=[(user['name'], uid) for uid, user in users.items()],
    description='User:'
)

date_selector = widgets.Dropdown(
    options=[(datetime.today() - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(31)],
    description='Date:'
)


In [ ]:
food_dropdown = widgets.Dropdown(options=food_df['name'].tolist(), description='Food:')
food_servings = widgets.BoundedFloatText(value=1, min=0, description='Servings:')
add_food_button = widgets.Button(description="Add Food")

food_log_output = widgets.Output()
food_log_list = []


In [ ]:
activity_dropdown = widgets.Dropdown(options=activity_df['activity'].unique().tolist(), description='Activity:')
activity_duration = widgets.BoundedIntText(value=30, min=1, description='Minutes:')
add_activity_button = widgets.Button(description="Add Activity")

activity_log_output = widgets.Output()
activity_log_list = []


In [ ]:
submit_button = widgets.Button(description="Calculate & Save", button_style='success')
daily_summary_output = widgets.Output()


In [ ]:
def on_add_food_clicked(b):
    food_log_list.append((food_dropdown.value, food_servings.value))
    with food_log_output:
        clear_output()
        print("🍎 Food Log:", food_log_list)

add_food_button.on_click(on_add_food_clicked)

def on_add_activity_clicked(b):
    activity_log_list.append((activity_dropdown.value, activity_duration.value))
    with activity_log_output:
        clear_output()
        print("🏃 Activity Log:", activity_log_list)

add_activity_button.on_click(on_add_activity_clicked)

def on_submit_clicked(b):
    user_id = user_selector.value
    date = date_selector.value
    log = log_daily_info(user_id, date, food_log_list, activity_log_list)

    with daily_summary_output:
        clear_output()
        print(f"📅 Daily Summary for {users[user_id]['name']} on {date}")
        print(json.dumps(log, indent=4))

    save_logs_to_file()  # Optional auto-save

submit_button.on_click(on_submit_clicked)


In [ ]:
# Widgets for entering user data
name_input = widgets.Text(description='Name:')
weight_input = widgets.BoundedFloatText(description='Weight (kg):', min=0)
height_input = widgets.BoundedFloatText(description='Height (cm):', min=0, max=1000)
age_input = widgets.BoundedIntText(description='Age:', min=1)
sex_input = widgets.Dropdown(options=['male', 'female'], description='Sex:')

add_user_button = widgets.Button(description="Add User", button_style='primary')
add_user_output = widgets.Output()


In [ ]:
import uuid  # For unique user IDs

def add_user(b):
    user_id = str(uuid.uuid4())
    users[user_id] = {
        'name': name_input.value,
        'weight': weight_input.value,
        'height': height_input.value,
        'age': age_input.value,
        'sex': sex_input.value
    }
    # Update dropdown options
    user_selector.options = [(user['name'], uid) for uid, user in users.items()]

    with add_user_output:
        clear_output()
        print(f"✅ Added user: {name_input.value} (ID: {user_id})")

add_user_button.on_click(add_user)


In [ ]:
display(widgets.VBox([
    widgets.HTML("<h4>➕ Add New User</h4>"),
    name_input, weight_input, height_input, age_input, sex_input,
    add_user_button, add_user_output
]))


In [ ]:
display(widgets.VBox([
    widgets.HBox([user_selector, date_selector]),

    widgets.HTML("<b>🍎 Add Food</b>"),
    widgets.HBox([food_dropdown, food_servings, add_food_button]),
    food_log_output,

    widgets.HTML("<b>🏃 Add Activity</b>"),
    widgets.HBox([activity_dropdown, activity_duration, add_activity_button]),
    activity_log_output,

    widgets.HTML("<hr><b>🧮 Calculate & Save Daily Log</b>"),
    submit_button,
    daily_summary_output
]))


✅ Daily logs saved to daily_logs.json
